In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [4]:
from zipfile import ZipFile 
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.transform import resize
import os
from matplotlib.image import imread
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

## Extract images from the Zip files

In [5]:
train_file = '/kaggle/input/dogs-vs-cats/train.zip'
test_file = '/kaggle/input/dogs-vs-cats/test1.zip'

with ZipFile(train_file, 'r') as zip: 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 
    
with ZipFile(test_file, 'r') as zip: 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

Extracting all the files now...
Done!
Extracting all the files now...
Done!


In [6]:
training_images = os.listdir('/kaggle/working/train')

## Loading and resizing the images

In [7]:
Label = []
Features = []

for image in training_images:
    Label.append(image[:3])
    img = imread('/kaggle/working/train/'+image)
    new_img = resize(img, (100, 100))
    Features.append(new_img.flatten())

In [ ]:
feats = np.array(Features) / 255.0
feats.shape

In [8]:
np.array(Features).shape

(25000, 30000)

In [9]:
labels = pd.DataFrame(Label)
labels[labels=='dog'] = 0
labels[labels=='cat'] = 1

labs = labels.values.astype(np.uint8)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(np.array(Features), labs, test_size=0.2)

In [ ]:
param_grid={'C':[0.1,1,10,100], 
            'gamma':[0.0001,0.001,0.1,1], 
            'kernel':['rbf','poly']} 
  
    
svc=SVC(probability=True) 
  
svm_classifier=GridSearchCV(svc,param_grid)
svm_classifier.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
preds = svm_classifier.predict(X_test)
preds

In [10]:
print('Accuracy:', accuracy_score(preds, y_test))

Accuracy: 0.6836
